# Dialのアルゴリズム（先週の講義資料，一部間違ってましたすみません）

In [30]:
import os
import numpy as np
import networkx as nx
import csv
import time
import pandas as pd
import copy
import math

## とりあえず今まで通り N, L, Q とダイクストラ法を書く

In [16]:
N = {} #ノードの情報
L = {} #リンクの情報

f = open("../GitHub/data_set/tntp_box/SiouxFalls_net.tntp", "r")
# f = open("../GitHub/data_set/tntp_box/ChicagoSketch_net.tntp", "r")
# f = open("../GitHub/data_set/tntp_box/Sydney_net.tntp", "r")

flag = False
i = 0
for line in f:# ここでファイルを一行ずつ読んでいる．一気に読まないので使用メモリが少ない（多分）
    if flag == True:
        link_parameter = line.split()# 一つの文字列を，スペースとか改行とかタブとかで分割する
    
        from_node = int(link_parameter[0])
        to_node   = int(link_parameter[1])
        ff_time   = float(link_parameter[4])
        capacity  = float(link_parameter[2])
        
        L[i+1] = {}
        L[i+1]["index"]     = i
        L[i+1]["from_node"] = from_node
        L[i+1]["to_node"]   = to_node
        L[i+1]["ff_time"]   = ff_time
        L[i+1]["capacity"]  = capacity
        
        if N.get(from_node):# Nの中に from_node があるなら
            N[from_node]['outlink_list'] = N[from_node]['outlink_list'] + [i+1]
        else:
            N[from_node] = {}
            N[from_node]["index"] = from_node-1
            N[from_node]['outlink_list'] = [i+1]
            N[from_node]['inlink_list']  = []
            
        if N.get(to_node):# Nの中に to_node があるなら
            N[to_node]['inlink_list'] = N[to_node]['inlink_list'] + [i+1]
        else:
            N[to_node] = {}
            N[to_node]["index"] = to_node-1
            N[to_node]['outlink_list'] = []
            N[to_node]['inlink_list']  = [i+1]
            
        i = i + 1
    
    
    if line[0] == "~":
        flag = True
    
    
f.close()
len(N)
print(N[1])

{'index': 0, 'outlink_list': [1, 2], 'inlink_list': [3, 5]}


In [1]:
Q = {}
zone_list = []

f = open("../GitHub/ProgrammingSeminar/datafile/SiouxFalls_trips.tntp", "r")

flag = False
for line in f:# ここでファイルを一行ずつ読んでいる．一気に読まないので使用メモリが少ない（多分）
    
    if flag == True:
        dataline = line.split()# 一つの文字列を，スペースとか改行とかタブとかで分割する
        if dataline == []:
            flag = False
        else:
            lista = copy.deepcopy(dataline)
            lista.reverse()
            length = len(lista)
            for i in range(length):
                if lista[i] == ":":
                    j = length - 1 - i
                    del dataline[j]                    
            
            for i in range(int(len(dataline)/2)):
                dem = float(dataline[2*i+1].rstrip(";"))
                Q[origin][int(dataline[2*i])] = dem

    if line[0] == "O":
        flag = True
        dataline = line.split()
        origin = int(dataline[1])
        zone_list = zone_list + [origin]
        Q[origin] = {}

f.close()
# print(Q[1])
# zone_list

FileNotFoundError: [Errno 2] No such file or directory: '../GitHub/ProgrammingSeminar/datafile/SiouxFalls_trips.tntp'

In [8]:
class heap:
    def __init__(self, node_list):# ここでの node_list は（コスト，ノード番号）の２要素タプルを想定．
        self.h_list = []
        length = len(node_list)
        for i in range(length):
            self.heappush(node_list[i])
            # print(self.h_list)
    
    
    def swap(self, j, k):
        self.h_list[j], self.h_list[k] = self.h_list[k], self.h_list[j]
    
    
    def heappush(self, item):
        self.h_list.append(item)
        length = len(self.h_list)
        son = length - 1
        while True:
            par = (son - 1)//2
            if par < 0:
                break
            elif self.h_list[son][0] < self.h_list[par][0]:# コストのみ見て並べ替え
                self.swap(son, par)
                son = par
            else:
                break

    
    def heappop(self):
        tail = self.h_list.pop(-1)
        if self.h_list == []:
            return tail
        
        # 最後尾要素を先頭へ
        pop_item       = self.h_list[0]
        self.h_list[0] = tail
        
        length = len(self.h_list)
        
        # 上からヒープ作り直し
        par = 0
        while True:
            son = par*2 + 1
            if son >= length:
                break
            elif son+1 == length:
                if self.h_list[son][0] < self.h_list[par][0]:
                    self.swap(son, par)
                    par = son
                else:
                    break
            else:
                if self.h_list[son][0] < self.h_list[son+1][0]:
                    if self.h_list[son][0] < self.h_list[par][0]:
                        self.swap(son, par)
                        par = son
                    else:
                        break
                else:
                    if self.h_list[son+1][0] < self.h_list[par][0]:
                        self.swap(son+1, par)
                        par = son + 1
                    else:
                        break
            
        return pop_item

In [9]:
def dijkstra(N, L, origin, link_cost):
    INF = 10 ** 9
    
    num_node = len(N)
    num_link = len(L)
    
    # 最短経路上の直前リンクを記憶する辞書
    pre_link = {origin:0}
    
    
    spl  = np.ones(shape=num_node)*INF#shortest path length
    flag = np.zeros(shape=num_node)
    
    hq = heap([(0, origin)]) # (distance, node)
    spl[N[origin]['index']]  = 0
    
    while hq.h_list:
        from_node = hq.heappop()[1] # 確定させるノードを pop
            
        if flag[N[from_node]['index']] == 2:
            continue #探索済ならスキップ
        else:
            for l in N[from_node]['outlink_list']:
                to_node = L[l]['to_node']
                tmp_cost = spl[N[from_node]['index']] + link_cost[L[l]['index']]
            
                if flag[N[to_node]['index']] == 2:
                    continue #探索済ならスキップ
                else:
                    if tmp_cost < spl[N[to_node]['index']]:
                        spl[N[to_node]['index']]  = tmp_cost
                        flag[N[to_node]['index']] = 1 #探索中に変更
                        hq.heappush((tmp_cost, to_node)) #ヒープに追加
                        pre_link[to_node] = l
                
            flag[N[from_node]['index']] = 2 #探索済に変更 
        
    return spl, pre_link

## ここから

In [1]:
def Dial_assignment(N, L, demand, zone_list, link_cost, origin):
    
    # リンクコストから pi を計算
    spl = dijkstra(N, L, origin, link_cost)[0]
    
    # リンク尤度(リスト)作成
    link_likeli = make_link_likeli(N, L, origin, link_cost, spl)
        
    # pi の順にノードを並び替え
    N_sorted = sort_N(N, spl)
    
    # リンク重み(辞書)作成
    link_weight = make_link_weight(N, L, origin, link_likeli, N_sorted)
       
    # 確率的リンクフロー（リスト）作成
    link_flow = make_link_flow(N, L, link_weight, N_sorted, demand)
      
    return link_flow
    
    
def make_link_likeli(N, L, origin, link_cost, spl):
    link_likeli = []
    for i in range(len(L)):
            
        #まずリンク前後のノードの pi を特定
        from_node = L[i+1]["from_node"]
        to_node = L[i+1]["to_node"]
        
        pi_from = spl[N[from_node]['index']]
        pi_to = spl[N[to_node]['index']]
        
        
        #pi_zeroの大小から尤度作成
        if pi_from < pi_to:
            likeli = math.exp(pi_to - pi_from - link_cost[i])
        else:
            likeli = 0
        link_likeli.append(likeli)
        
    return link_likeli
    
    
def sort_N(N, spl):
        
    # 新たにNを作って"pi_zero"の要素を追加
    N_new = copy.deepcopy(N)
    for i in range(len(N)):
        try:
            N_new[i+1]["pi"] = spl[N[i+1]['index']]
        except KeyError:
            # print("False node :", i+1)
            N_new[i+1] = {'index': i, 'pi_index': -1, 'outlink_list': [], 'inlink_list': [], "pi": 10**9}
      
    # pi の小さい順にソート
    N_sorted_list = sorted(N_new.items(), key=lambda x:x[1]["pi"])
    N_sorted = {}
    for i in N_sorted_list:
        N_sorted[i[0]] = i[1]
            
    return N_sorted
    
    
def make_link_weight(N, L, origin, link_likeli, N_sorted):
    keys = list(N_sorted.keys())
       
    link_weight = {}
    for i in range(len(L)):
        link_weight[i+1] = 0
        
    #起点から近い順にfor文．ただしFalse nodeまでたどり着いたらbreak
    for i in keys:
        if N_sorted[i]["pi"] > 10**8:
            break
    
        if i == origin:
            prod = 1
        else:
            prod = 0
            for k in N[i]['inlink_list']:
                prod = prod + link_weight[k]
            
        for j in N[i]['outlink_list']:
            link_weight[j] = link_likeli[L[j]["index"]]*prod
                
    return link_weight
    
    
def make_link_flow(N, L, link_weight, N_sorted, demand, zone_list):
    keys = list(N_sorted.keys())
    link_flow = np.zeros(shape=len(L))
        
    #起点から遠い順にfor文．False nodeは最初にスキップ
    rev_keys = reversed(keys)
    for i in rev_keys:
        if N_sorted[i]["pi"] > 10**8:
            pass
        else:
            #このノードにくる合計フローを計算
            outflow = 0
            for j in N[i]['outlink_list']:
                outflow = outflow + link_flow[j-1]
                
            #終点ノードの場合のみ需要も加える
            if i in zone_list:
                prod_1 = demand[i] + outflow
            else:
                prod_1 = outflow
                
            #このノードにくる合計リンク尤度を計算
            inweight = 0
            for j in N[i]['inlink_list']:
                inweight = inweight + link_weight[j]
                
            #合計フローをリンク尤度で案分
            for j in N[i]['inlink_list']:
                if inweight == 0:
                    inweight = 1
                link_flow[j-1] = prod_1*link_weight[j]/inweight
        
    return link_flow

In [48]:
t_0 = np.array([L[i+1]["ff_time"] for i in range(len(L))])
flow = np.zeros(shape=len(L))

# 全起点分のフローを足し算
for i in zone_list:
    linkflow = Dial_assignment(N, L, Q[1], zone_list, t_0, i)
    flow = flow + linkflow

print(flow)

[ 3615.68643117  6911.40664374  1227.08103513 10488.59335626
   500.01203979 13658.38197485 10015.97859123  4498.68610311
 16325.23774899  7390.55555212 12368.84534352 13114.66794962
  9936.51610825  1699.98796021 11368.25197549 22741.75151553
  9626.07124985 10004.1520374  10606.73014535  8246.72726342
   541.18532642 12994.83302155  3726.53967691   684.6745736
 12596.49080813  9730.00362397  7486.25879557  4541.98364817
 10018.72955929   367.39290027  5387.25208584 13961.89314324
  3461.75481978  7655.15997389  6364.27985902  8308.83075089
 10527.70522536  7723.08242426  8349.9513818  10480.41492417
  6055.29950917  3629.8903203   9999.07284257  4769.32596629
 11560.96259621  8234.12235421  9736.97841777 17352.6745819
 13359.1952406   5161.81496841   634.23715144 15551.7854117
 10921.82146171 14583.49602383 10245.31521613  8490.2564428
 11149.07436403 14181.25588398  2124.93250172 11753.10067695
  3372.47869181  2339.39886946  4988.01696738  4622.99503041
 10928.80884502  6914.242782

In [49]:
# 参考：all or nothing 配分の結果

def all_or_nothing(N, L, Q, zone_list, link_cost):
    y = np.zeros(shape=len(L))
    
    # 全ての起点 i について順番に考える
    for i in zone_list:
        pre_link = dijkstra(N, L, i, link_cost)[1]
        
        # すべての終点 j についてさらに順番に考える
        for j in zone_list:
            link_set = []
            node = j
            
            # i から j への最短経路上のリンクを取得
            while True:
                link = pre_link[node]
                if link == 0:
                    break
                else:
                    link_set = link_set + [link]
                    node = L[pre_link[node]]["from_node"]
            
            # 最短経路上に需要を全て賦課
            q = Q[i][j]
            for k in link_set:
                y[L[k]["index"]] = y[L[k]["index"]] + q
    
    return y

t_0 = np.array([L[i+1]["ff_time"] for i in range(len(L))])
all_or_nothing(N, L, Q, zone_list, t_0)

array([ 3800.,  6000.,  3800.,  6600.,  6000.,  8400.,  8600.,  9400.,
       13700.,  5700., 14500., 10200.,  7000.,  6600., 11000., 16800.,
        8000., 13100., 17600.,  8000.,   800., 14700.,  7000.,   800.,
       17000., 17100., 21300., 11200., 28800.,     0.,  6000., 19600.,
       12100., 16900.,  7600., 13100., 12000., 12100., 10800., 14600.,
        8100., 10500., 13600., 10800., 18400., 20200., 15500., 28100.,
       26700., 14700.,     0., 26700., 21900., 13100., 14800., 12100.,
       18400., 21900.,  4500., 12100.,  4500.,  4400.,  5300.,  4300.,
       13000., 12800., 25200.,  5300., 13000.,  5800.,  5500., 10800.,
        5700., 10800., 12700.,  5700.])

## オーダーは同じだから多分あってんじゃね？